In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [9]:
df_raw = pd.read_csv("data.csv")

In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load your raw dataset (assuming it's in a DataFrame called df_raw)
df_raw['TransactionStartTime'] = pd.to_datetime(df_raw['TransactionStartTime'])

# Define the snapshot date
snapshot_date = df_raw['TransactionStartTime'].max() + pd.Timedelta(days=1)

# Group by CustomerId and compute RFM
rfm = df_raw.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (snapshot_date - x.max()).days,  # Recency
    'TransactionId': 'count',                                          # Frequency
    'Amount': 'sum'                                                    # Monetary
}).rename(columns={
    'TransactionStartTime': 'Recency',
    'TransactionId': 'Frequency',
    'Amount': 'Monetary'
}).reset_index()


In [11]:
# Scale the RFM features
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# Apply KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
rfm['cluster'] = kmeans.fit_predict(rfm_scaled)


In [12]:
# View cluster characteristics
cluster_summary = rfm.groupby('cluster')[['Recency', 'Frequency', 'Monetary']].mean()
print(cluster_summary)

# Assume the high-risk group has high Recency, low Frequency and low Monetary
# Sort by these criteria to identify the risky cluster
high_risk_cluster = cluster_summary.sort_values(
    by=['Recency', 'Frequency', 'Monetary'], 
    ascending=[False, True, True]
).index[0]

# Create binary label
rfm['is_high_risk'] = (rfm['cluster'] == high_risk_cluster).astype(int)


           Recency    Frequency      Monetary
cluster                                      
0        61.859846     7.726699  8.172379e+04
1        29.000000  4091.000000 -1.049000e+08
2        12.716076    34.807692  2.726546e+05


In [13]:
df_processed=pd.read_csv('preprocess.csv')

In [14]:
# Merge is_high_risk into your processed dataset
df_final = df_processed.merge(rfm[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')

# If needed, fill missing values (e.g., if some CustomerIds didn’t appear in RFM)
df_final['is_high_risk'] = df_final['is_high_risk'].fillna(0).astype(int)


In [17]:
df_final.head(10)

,log_amount,log_value,TransactionHour,TransactionDay,TransactionMonth,TransactionYear,amount_sum,amount_mean,amount_std,amount_count,...,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4,CustomerId,is_high_risk
0,-0.023705,-0.027186,-2.155530,-0.100739,0.848684,-0.994246,0.170118,-0.067623,-0.168551,-0.311831,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,CustomerId_4406,0
1,-1.854650,-1.850929,-2.155530,-0.100739,0.848684,-0.994246,0.170118,-0.067623,-0.168551,-0.311831,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,CustomerId_4406,0
2,-0.351658,-0.353849,-2.155530,-0.100739,0.848684,-0.994246,0.165122,-0.072568,-0.202748,-0.444993,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,CustomerId_4683,1
3,1.395279,1.426886,-1.949214,-0.100739,0.848684,-0.994246,0.175567,-0.008155,-0.009754,-0.404020,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,CustomerId_988,0
4,-0.231950,-0.234613,-1.949214,-0.100739,0.848684,-0.994246,0.175567,-0.008155,-0.009754,-0.404020,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,CustomerId_988,0
5,0.304484,0.299712,-1.949214,-0.100739,0.848684,-0.994246,0.165168,-0.055062,0.000000,-0.446132,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,CustomerId_1432,1
6,1.066876,1.059105,-1.949214,-0.100739,0.848684,-0.994246,0.169360,-0.040815,-0.140992,-0.414264,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,CustomerId_2858,0
7,-0.351658,-0.353849,-1.949214,-0.100739,0.848684,-0.994246,0.169360,-0.040815,-0.140992,-0.414264,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,CustomerId_2858,0
8,-0.351658,-0.353849,-1.742898,-0.100739,0.848684,-0.994246,0.165446,-0.054887,-0.165175,-0.442717,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,CustomerId_598,1
9,-0.265428,-0.267959,-1.742898,-0.100739,0.848684,-0.994246,0.165709,-0.055415,-0.181262,-0.439303,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,CustomerId_1053,1


In [18]:
df_final.to_csv("df_final_task4.csv", index=False)


In [19]:
df_processed.head()

,log_amount,log_value,TransactionHour,TransactionDay,TransactionMonth,TransactionYear,amount_sum,amount_mean,amount_std,amount_count,...,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4,CustomerId
0,-0.023705,-0.027186,-2.155530,-0.100739,0.848684,-0.994246,0.170118,-0.067623,-0.168551,-0.311831,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,CustomerId_4406
1,-1.854650,-1.850929,-2.155530,-0.100739,0.848684,-0.994246,0.170118,-0.067623,-0.168551,-0.311831,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,CustomerId_4406
2,-0.351658,-0.353849,-2.155530,-0.100739,0.848684,-0.994246,0.165122,-0.072568,-0.202748,-0.444993,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,CustomerId_4683
3,1.395279,1.426886,-1.949214,-0.100739,0.848684,-0.994246,0.175567,-0.008155,-0.009754,-0.404020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,CustomerId_988
4,-0.231950,-0.234613,-1.949214,-0.100739,0.848684,-0.994246,0.175567,-0.008155,-0.009754,-0.404020,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,CustomerId_988
